In [1]:
import cv2
import numpy as np
import os
import yaml
from yaml.loader import SafeLoader

In [2]:
labels=['car', 'truck', 'pedestrian', 'bicyclist', 'light']

In [3]:
yolo=cv2.dnn.readNetFromONNX("C:/Users/varun/OneDrive/Desktop/Yolo/model/Model4/weights/best.onnx")
yolo.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
yolo.setPreferableTarget(cv2.dnn.DNN_TARGET_CPU)

In [4]:
img=cv2.imread('C:/Users/varun/OneDrive/Desktop/yolo/images/traffic.jpg')
image=img.copy()
row, col, d=image.shape

max_rc=max(row,col)
input_image=np.zeros((max_rc,max_rc,3),dtype=np.uint8)

input_image[:row,:col] = image

INPUT_WH_YOLO = 640 
blob = cv2.dnn.blobFromImage(input_image,1/255,(INPUT_WH_YOLO,INPUT_WH_YOLO),swapRB=True,crop=False)
 
yolo.setInput(blob) 
preds = yolo.forward()

In [5]:
preds

array([[[2.59312820e+00, 5.06073189e+00, 9.35577202e+00, ...,
         5.23483753e-02, 1.44058699e-02, 5.74992836e-01],
        [8.48691177e+00, 4.79698181e+00, 1.69785233e+01, ...,
         3.91665027e-02, 1.56525746e-02, 5.96078694e-01],
        [1.72228909e+01, 3.88028908e+00, 2.25369720e+01, ...,
         4.39776927e-02, 1.94947924e-02, 7.21913874e-01],
        ...,
        [5.73388916e+02, 6.12291748e+02, 1.96802750e+02, ...,
         1.09408945e-01, 9.88907591e-02, 1.30682051e-01],
        [5.92450562e+02, 6.09658936e+02, 1.53049301e+02, ...,
         2.15494037e-01, 1.61423355e-01, 1.67503312e-01],
        [6.15962585e+02, 6.10648376e+02, 1.88257919e+02, ...,
         2.54150867e-01, 2.27203086e-01, 2.34153315e-01]]], dtype=float32)

In [6]:
preds.shape

(1, 25200, 10)

In [7]:
detections=preds[0]
boxes=[]
classes = []
confidences = []

image_w,image_h=input_image.shape[:2]
y_factor = image_h/INPUT_WH_YOLO 
x_factor = image_w/INPUT_WH_YOLO

for i in range(len(detections)):
    row = detections[i]
    confidence = row[4]
    if confidence > 0.4:
        class_score = row[5:].max()
        class_id = row[5:].argmax() 
        
        if class_score>0.25:
            cx,cy,w,h=row[0:4]
        left = int((cx-0.5*w)*x_factor)
        top = int((cy-0.5*h)*y_factor)
        width = int(w*x_factor)
        height = int(h*y_factor)
        
        box=np.array([left,top,width,height])
        
        confidences.append(confidence)
        boxes.append(box)
        classes.append(class_id)
        
        
        
boxes_np=np.array(boxes).tolist()
confidences_np=np.array(confidences).tolist()

index=cv2.dnn.NMSBoxes(boxes_np,confidences,0.25,0.45).flatten()

In [8]:
for i in index:
    x,y,w,h=boxes_np[i]
    bb_conf=int(confidences_np[i]*100)
    classes_id=classes[i]
    class_name=labels[classes_id]
    
    text=f'{class_name}:{bb_conf}%'
    
    cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
    cv2.rectangle(image,(x,y-30),(x+w,y),(255,255,255),-1)
    
    cv2.putText(image,text,(x,y-10),cv2.FONT_HERSHEY_PLAIN,0.7,(0,0,0),1)

In [9]:
cv2.imshow('original',img)
cv2.imshow('yolo_prediction',image)
cv2.waitKey(0)
cv2.destroyAllWindows()